In [ ]:
import chardet
from bs4 import BeautifulSoup

def html_to_text(html_path):
    with open(html_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    with open(html_path, 'r', encoding=encoding) as file:
        soup = BeautifulSoup(file, 'html.parser')
        text = soup.get_text()
    return text

# Загрузка и конвертация книги
html_path = 'D:\\Study\\3grade\\AI\\Labs3\\Аврамeнко.html'
text = html_to_text(html_path)

# Сохранение текста в файл
with open('bookA.txt', 'w', encoding='utf-8') as file:
    file.write(text)

In [1]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Input, Dense
from tensorflow.keras.preprocessing.text import Tokenizer

## Предобработка текста

In [2]:
file_path = 'D:\\Study\\3grade\\AI\\Labs3\\bookA.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

text = text.replace('\n', ' ').replace('\r', '')

print(text[:1000])

часть первая ДОРОГА В НИКУДА    Глава I МИРНАЯ ПОТАСОВКА   Не выпуская из левой руки уздечку, правой рукой Карсидар пригнул ветви буйно разросшегося на опушке леса кустарника. Он с сомнением посмотрел на стоявший у развилки дом, неодобрительно покачал головой и даже слегка поморщился.                - Ристо, как ты считаешь, мне стоит туда идти?                В ответ Ристо шумно вздохнул, потянул ноздрями воздух и негромко, но явно утвердительно рявкнул: "Гррххх...".                Оно и понятно, угрюмо подумал Карсидар. Чего этой скотине надо для полного счастья? Тёплое стойло на ночь, ведро студёной воды, побольше овса в корыте - вот и все его нехитрые требования. Ему не нужно быть уверенным в спутнике почти как в самом себе, не нужно настораживаться при каждом подозрительном шорохе, не нужно спать чутко, просыпаясь от малейшего шума в ночи. Да и, собственно, с какой стати - ведь не за его же голову назначена награда в тридцать два жуда чистым золотом. За голову Ристо никто не дал б

## Однослойная и многослойная LSTM по символам

### Токенизация

In [3]:
chars = sorted(list(set(text)))
char_to_idx = {ch: idx for idx, ch in enumerate(chars)}
idx_to_char = {idx: ch for idx, ch in enumerate(chars)}

encoded_text = [char_to_idx[ch] for ch in text]

print(encoded_text[:100])

[72, 49, 66, 67, 77, 0, 64, 54, 65, 51, 49, 80, 0, 22, 32, 34, 32, 21, 18, 0, 20, 0, 31, 26, 28, 37, 22, 18, 0, 17, 17, 0, 21, 60, 49, 51, 49, 0, 14, 0, 30, 26, 34, 31, 18, 48, 0, 33, 32, 36, 18, 35, 32, 20, 28, 18, 0, 17, 17, 31, 54, 0, 51, 76, 64, 68, 66, 59, 49, 80, 0, 57, 56, 0, 60, 54, 51, 63, 58, 0, 65, 68, 59, 57, 0, 68, 56, 53, 54, 72, 59, 68, 5, 0, 64, 65, 49, 51, 63, 58]


### Обрабатываем данные для обучения

In [4]:
seq_length = 100
step_size = 1

sequences = []
next_chars = []

for i in range(0, len(encoded_text) - seq_length, step_size):
    sequences.append(encoded_text[i: i + seq_length])
    next_chars.append(encoded_text[i + seq_length])

X = np.zeros((len(sequences), seq_length, len(chars)), dtype=bool)
Y = np.zeros((len(sequences), len(chars)), dtype=bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        X[i, t, char] = 1
    Y[i, next_chars[i]] = 1

print(f'Количество обучающих последовательностей: {len(sequences)}')

Количество обучающих последовательностей: 977054


### Однослойная модель LSTM по символам

In [5]:
model = Sequential()
model.add(Input(shape=(seq_length, len(chars))))
model.add(LSTM(128))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │       108,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 83)             │        10,707 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 119,251 (465.82 KB)

 Trainable params: 119,251 (465.82 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
model.fit(X, Y, batch_size=128, epochs=20)

Epoch 1/20
7634/7634 ━━━━━━━━━━━━━━━━━━━━ 692s 90ms/step - loss: 2.4581
Epoch 2/20
7634/7634 ━━━━━━━━━━━━━━━━━━━━ 678s 89ms/step - loss: 1.9087
Epoch 3/20
7634/7634 ━━━━━━━━━━━━━━━━━━━━ 683s 89ms/step - loss: 1.7581
Epoch 4/20
7634/7634 ━━━━━━━━━━━━━━━━━━━━ 687s 90ms/step - loss: 1.6669
Epoch 5/20
7634/7634 ━━━━━━━━━━━━━━━━━━━━ 701s 92ms/step - loss: 1.6079
Epoch 6/20
7634/7634 ━━━━━━━━━━━━━━━━━━━━ 710s 93ms/step - loss: 1.5641
Epoch 7/20
7634/7634 ━━━━━━━━━━━━━━━━━━━━ 860s 113ms/step - loss: 1.5263
Epoch 8/20
7634/7634 ━━━━━━━━━━━━━━━━━━━━ 867s 114ms/step - loss: 1.5043
Epoch 9/20
7634/7634 ━━━━━━━━━━━━━━━━━━━━ 877s 115ms/step - loss: 1.4811
Epoch 10/20
7634/7634 ━━━━━━━━━━━━━━━━━━━━ 871s 114ms/step - loss: 1.4636
Epoch 11/20
7634/7634 ━━━━━━━━━━━━━━━━━━━━ 889s 116ms/step - loss: 1.4481
Epoch 12/20
7634/7634 ━━━━━━━━━━━━━━━━━━━━ 869s 114ms/step - loss: 1.4320
Epoch 13/20
7634/7634 ━━━━━━━━━━━━━━━━━━━━ 871s 114ms/step - loss: 1.4290
Epoch 14/20
7634/7634 ━━━━━━━━━━━━━━━━━━━━ 889s 116ms

### Многослойная модель LSTM по символам

In [7]:
model_multi = Sequential()
model_multi.add(Input(shape=(seq_length, len(chars))))
model_multi.add(LSTM(128, return_sequences=True))
model_multi.add(LSTM(128))
model_multi.add(Dense(len(chars), activation='softmax'))

model_multi.compile(loss='categorical_crossentropy', optimizer='adam')

model_multi.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 100, 128)       │       108,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 83)             │        10,707 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 250,835 (979.82 KB)

 Trainable params: 250,835 (979.82 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
model_multi.fit(X, Y, batch_size=128, epochs=10)

Epoch 1/10
7634/7634 ━━━━━━━━━━━━━━━━━━━━ 1987s 260ms/step - loss: 2.4514
Epoch 2/10
7634/7634 ━━━━━━━━━━━━━━━━━━━━ 2019s 264ms/step - loss: 1.7971
Epoch 3/10
7634/7634 ━━━━━━━━━━━━━━━━━━━━ 2100s 275ms/step - loss: 1.6161
Epoch 4/10
7634/7634 ━━━━━━━━━━━━━━━━━━━━ 2103s 276ms/step - loss: 1.5286
Epoch 5/10
7634/7634 ━━━━━━━━━━━━━━━━━━━━ 2082s 273ms/step - loss: 1.4682
Epoch 6/10
7634/7634 ━━━━━━━━━━━━━━━━━━━━ 2136s 280ms/step - loss: 1.4304
Epoch 7/10
7634/7634 ━━━━━━━━━━━━━━━━━━━━ 2182s 286ms/step - loss: 1.3959
Epoch 8/10
7634/7634 ━━━━━━━━━━━━━━━━━━━━ 2225s 291ms/step - loss: 1.3717
Epoch 9/10
7634/7634 ━━━━━━━━━━━━━━━━━━━━ 2326s 305ms/step - loss: 1.3513
Epoch 10/10
7634/7634 ━━━━━━━━━━━━━━━━━━━━ 2413s 316ms/step - loss: 1.3296


## Однослойная и многослойная LSTM по словам

### Токенизация по словам

In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

encoded_text_words = tokenizer.texts_to_sequences([text])[0]

word_to_idx = tokenizer.word_index
idx_to_word = {v: k for k, v in word_to_idx.items()}

print(encoded_text_words[:100])

[635, 4558, 1126, 3, 2808, 4, 665, 6445, 3428, 4559, 1313, 6446, 24, 1444, 173, 1445, 904, 320, 11, 10561, 6447, 10562, 10563, 5, 2809, 1212, 10564, 9, 7, 4560, 278, 5, 2369, 26, 6448, 309, 10565, 816, 202, 1, 56, 203, 2031, 4, 147, 10, 15, 2032, 51, 345, 398, 700, 298, 817, 147, 4561, 399, 2370, 10566, 1446, 1, 10567, 17, 287, 1771, 1447, 6449, 10568, 1, 967, 1042, 144, 11, 87, 273, 10569, 125, 67, 4562, 2371, 4563, 10570, 5, 728, 10571, 3429, 487, 1772, 10572, 3, 6450, 31, 1, 72, 12, 10573, 10574, 42, 2, 415]


### Готовим данные

In [10]:
import numpy as np

sequence_length = 10
step_size = 1

sequences_words = []
next_words = []

for i in range(0, len(encoded_text_words) - sequence_length, step_size):
    sequences_words.append(encoded_text_words[i: i + sequence_length])
    next_words.append(encoded_text_words[i + sequence_length])

X_words_arr = np.array(sequences_words)
Y_words_arr = np.array(next_words)

print(f'Количество обучающих последовательностей: {len(sequences_words)}')

Количество обучающих последовательностей: 138238


### Модель однослойной LSTM по словам

In [11]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

embedding_dim = 100

model_words = Sequential()
model_words.add(Embedding(input_dim=len(word_to_idx) + 1, output_dim=embedding_dim))
model_words.add(LSTM(128))
model_words.add(Dense(len(word_to_idx) + 1, activation='softmax'))

model_words.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

model_words.build(input_shape=(None, sequence_length))
model_words.summary()

X_words_arr = np.array(sequences_words)
Y_words_arr = np.array(next_words)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 10, 100)        │     2,584,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 128)            │       117,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 25845)          │     3,334,005 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,035,753 (23.02 MB)

 Trainable params: 6,035,753 (23.02 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
batch_size = 128
model_words.fit(X_words_arr, Y_words_arr, batch_size=batch_size, epochs=20)

Epoch 1/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 86s 78ms/step - loss: 8.5286
Epoch 2/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 85s 78ms/step - loss: 7.7591
Epoch 3/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 85s 79ms/step - loss: 7.2962
Epoch 4/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 85s 79ms/step - loss: 6.8649
Epoch 5/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 85s 79ms/step - loss: 6.4302
Epoch 6/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 85s 79ms/step - loss: 5.9621
Epoch 7/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 85s 79ms/step - loss: 5.5335
Epoch 8/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 85s 79ms/step - loss: 5.1162
Epoch 9/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 85s 78ms/step - loss: 4.7014
Epoch 10/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 85s 79ms/step - loss: 4.2957
Epoch 11/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 85s 79ms/step - loss: 3.9049
Epoch 12/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 85s 79ms/step - loss: 3.5471
Epoch 13/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 85s 79ms/step - loss: 3.2151
Epoch 14/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 85s 79ms/step - loss: 2.9238
E

### Многослойная модель LSTM по словам

In [13]:
embedding_dim = 100

model_multi_words = Sequential()
model_multi_words.add(Embedding(input_dim=len(word_to_idx) + 1, output_dim=embedding_dim))
model_multi_words.add(LSTM(128, return_sequences=True))
model_multi_words.add(LSTM(128))
model_multi_words.add(Dense(len(word_to_idx) + 1, activation='softmax'))

model_multi_words.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

model_multi_words.build(input_shape=(None, sequence_length))
model_multi_words.summary()

X_words_arr = np.array(sequences_words)
Y_words_arr = np.array(next_words)

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 10, 100)        │     2,584,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 10, 128)        │       117,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 25845)          │     3,334,005 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,167,337 (23.53 MB)

 Trainable params: 6,167,337 (23.53 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
batch_size = 128
model_multi_words.fit(X_words_arr, Y_words_arr, batch_size=batch_size, epochs=20)

Epoch 1/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 100s 90ms/step - loss: 8.5557
Epoch 2/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 97s 90ms/step - loss: 7.8645
Epoch 3/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 97s 90ms/step - loss: 7.6026
Epoch 4/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 97s 90ms/step - loss: 7.3542
Epoch 5/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 97s 90ms/step - loss: 7.1299
Epoch 6/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 97s 90ms/step - loss: 6.8843
Epoch 7/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 97s 90ms/step - loss: 6.6457
Epoch 8/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 97s 90ms/step - loss: 6.3995
Epoch 9/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 97s 90ms/step - loss: 6.1539
Epoch 10/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 97s 90ms/step - loss: 5.9139
Epoch 11/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 97s 90ms/step - loss: 5.6984
Epoch 12/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 97s 89ms/step - loss: 5.4648
Epoch 13/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 97s 90ms/step - loss: 5.2149
Epoch 14/20
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 97s 89ms/step - loss: 4.9554


## BPE

### Токенизация

In [15]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers

tokenizer_bpe = Tokenizer(models.BPE())
tokenizer_bpe.pre_tokenizer = pre_tokenizers.ByteLevel()

trainer = trainers.BpeTrainer(vocab_size=30000, special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"])
tokenizer_bpe.train([file_path], trainer)

encoded_text_bpe = tokenizer_bpe.encode(text).ids

print(encoded_text_bpe[:100])

[2195, 12010, 22569, 27813, 230, 22020, 15111, 26048, 990, 116, 2897, 6845, 14824, 26888, 2567, 11219, 24781, 25082, 11536, 990, 116, 2119, 19958, 274, 5094, 1127, 5190, 9, 3607, 1716, 208, 26865, 16869, 11038, 147, 22592, 5940, 163, 8186, 4498, 5695, 20738, 11, 538, 113, 12639, 1570, 163, 8282, 162, 15952, 742, 9, 6248, 25096, 3299, 1262, 126, 612, 1245, 6831, 11, 104, 104, 138, 116, 10, 980, 9, 231, 291, 6589, 9, 594, 1861, 2020, 3075, 17, 104, 104, 138, 116, 511, 1497, 980, 12853, 1934, 9, 5850, 10899, 23477, 2226, 126, 13519, 281, 9, 376, 1640, 6152, 5206]


### Подготовка данных

In [16]:
seq_length_bpe = 100
step_size = 1

sequences_bpe = []
next_bpe = []

for i in range(0, len(encoded_text_bpe) - seq_length_bpe, step_size):
    sequences_bpe.append(encoded_text_bpe[i: i + seq_length_bpe])
    next_bpe.append(encoded_text_bpe[i + seq_length_bpe])

X_bpe = np.array(sequences_bpe)
Y_bpe = np.array(next_bpe)

print(f'Количество обучающих последовательностей: {len(sequences_bpe)}')

Количество обучающих последовательностей: 204812


### Модель однослойной LSTM BPE

In [17]:
embedding_dim = 100
vocab_size = tokenizer_bpe.get_vocab_size()

model_bpe = Sequential()
model_bpe.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
model_bpe.add(LSTM(128))
model_bpe.add(Dense(vocab_size, activation='softmax'))

model_bpe.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

model_bpe.build(input_shape=(None, seq_length_bpe))
model_bpe.summary()

X_bpe = np.array(sequences_bpe)
Y_bpe = np.array(next_bpe)

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 100, 100)       │     3,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 128)            │       117,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 30000)          │     3,870,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,987,248 (26.65 MB)

 Trainable params: 6,987,248 (26.65 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model_bpe.fit(X_bpe, Y_bpe, batch_size=128, epochs=20)

Epoch 1/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 356s 222ms/step - loss: 7.4301
Epoch 2/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 334s 208ms/step - loss: 6.0508
Epoch 3/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 330s 206ms/step - loss: 5.5687
Epoch 4/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 329s 206ms/step - loss: 5.1896
Epoch 5/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 326s 204ms/step - loss: 4.8306
Epoch 6/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 328s 205ms/step - loss: 4.4830
Epoch 7/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 329s 205ms/step - loss: 4.1728
Epoch 8/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 337s 211ms/step - loss: 3.8798
Epoch 9/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 314s 196ms/step - loss: 3.5817
Epoch 10/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 310s 193ms/step - loss: 3.3071
Epoch 11/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 299s 187ms/step - loss: 3.0682
Epoch 12/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 298s 186ms/step - loss: 2.8466
Epoch 13/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 298s 186ms/step - loss: 2.6366
Epoch 14/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 297s

### Многослойная модель LSTM BPE

In [19]:
model_multi_bpe = Sequential()
model_multi_bpe.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
model_multi_bpe.add(LSTM(128, return_sequences=True))
model_multi_bpe.add(LSTM(128))
model_multi_bpe.add(Dense(vocab_size, activation='softmax'))

model_multi_bpe.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

model_multi_bpe.build(input_shape=(None, seq_length_bpe))
model_multi_bpe.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 100, 100)       │     3,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 100, 128)       │       117,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 30000)          │     3,870,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,118,832 (27.16 MB)

 Trainable params: 7,118,832 (27.16 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
model_multi_bpe.fit(X_bpe, Y_bpe, batch_size=128, epochs=20)

Epoch 1/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 503s 313ms/step - loss: 7.5672
Epoch 2/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 506s 316ms/step - loss: 6.3145
Epoch 3/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 499s 312ms/step - loss: 5.9457
Epoch 4/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 503s 314ms/step - loss: 5.6687
Epoch 5/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 505s 315ms/step - loss: 5.4118
Epoch 6/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 504s 315ms/step - loss: 5.1925
Epoch 7/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 513s 321ms/step - loss: 4.9648
Epoch 8/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 514s 321ms/step - loss: 4.7588
Epoch 9/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 517s 323ms/step - loss: 4.5384
Epoch 10/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 517s 323ms/step - loss: 4.3255
Epoch 11/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 522s 326ms/step - loss: 4.1317
Epoch 12/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 523s 326ms/step - loss: 3.9496
Epoch 13/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 544s 340ms/step - loss: 3.7620
Epoch 14/20
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 585s

## Оценка

In [21]:
# Оценка моделей с посимвольной токенизацией
loss_char = model.evaluate(X, Y)
loss_char_multi = model_multi.evaluate(X, Y)

# Оценка моделей с токенизацией по словам
loss_word = model_words.evaluate(X_words_arr, Y_words_arr)
loss_word_multi = model_multi_words.evaluate(X_words_arr, Y_words_arr)

# Оценка моделей с BPE токенизацией
loss_bpe = model_bpe.evaluate(X_bpe, Y_bpe)
loss_bpe_multi = model_multi_bpe.evaluate(X_bpe, Y_bpe)

print("Loss (Char-Level, Single Layer):", loss_char)
print("Loss (Char-Level, Multi Layer):", loss_char_multi)
print("Loss (Word-Level, Single Layer):", loss_word)
print("Loss (Word-Level, Multi Layer):", loss_word_multi)
print("Loss (BPE, Single Layer):", loss_bpe)
print("Loss (BPE, Multi Layer):", loss_bpe_multi)

30533/30533 ━━━━━━━━━━━━━━━━━━━━ 820s 27ms/step - loss: 1.3452
30533/30533 ━━━━━━━━━━━━━━━━━━━━ 1533s 50ms/step - loss: 1.2897
4320/4320 ━━━━━━━━━━━━━━━━━━━━ 38s 9ms/step - loss: 1.4976
4320/4320 ━━━━━━━━━━━━━━━━━━━━ 49s 11ms/step - loss: 3.4618
6401/6401 ━━━━━━━━━━━━━━━━━━━━ 198s 31ms/step - loss: 1.5353
6401/6401 ━━━━━━━━━━━━━━━━━━━━ 353s 55ms/step - loss: 2.6308
Loss (Char-Level, Single Layer): 1.3566956520080566
Loss (Char-Level, Multi Layer): 1.301733136177063
Loss (Word-Level, Single Layer): 1.4417921304702759
Loss (Word-Level, Multi Layer): 3.456880807876587
Loss (BPE, Single Layer): 1.5088529586791992
Loss (BPE, Multi Layer): 2.6373353004455566


## Генерация

### Посимвольная

In [22]:
def generate_text(model, start_text, char_to_idx, idx_to_char, length=400, temperature=1.0):
    # Преобразование стартового текста в индексы
    input_seq = [char_to_idx[char] for char in start_text]
    generated_text = start_text

    for _ in range(length):
        # Преобразование последовательности в one-hot формат
        x_pred = np.zeros((1, len(input_seq), len(char_to_idx)))
        for t, char_idx in enumerate(input_seq):
            x_pred[0, t, char_idx] = 1.0

        # Предсказание следующего символа
        preds = model.predict(x_pred, verbose=0)[0]
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)

        next_index = np.random.choice(range(len(preds)), p=preds)
        next_char = idx_to_char[next_index]

        # Добавление предсказанного символа в результат
        generated_text += next_char
        input_seq.append(next_index)
        input_seq = input_seq[1:]

    return generated_text

start_text = "Однажды"

# Генерация текста для однослойной модели
print("Generated text (Char-Level, Single Layer):")
print(generate_text(model, start_text, char_to_idx, idx_to_char))

# Генерация текста для многослойной модели
print("\nGenerated text (Char-Level, Multi Layer):")
print(generate_text(model_multi, start_text, char_to_idx, idx_to_char))

Generated text (Char-Level, Single Layer):
Однаждыйны штучки не сышку ему, почему смытая церкольюцы измлик. Ссазали постаствеены наш начиской уже решилишь, хватиту здавил он никто Михайла. Святствов.                                                                                                                                                                                                                                                           

Generated text (Char-Level, Multi Layer):
Однажды так, а грозой., а как и тщастью. Ненагда), которым. НачалиЯ. Не пристался. Вочтья венец, повездоннимы. - Спасосми, получишь, входяшь - бы всяко, допачилам. Прислусь, способемовилась, обязаенстое была твой что-то остался. Некачу!.. Брит. С вастойно денстоцах, более вружался? Присупслив... втоснул.                                                                                                      


### По словам

In [26]:
def generate_text_words(model, start_text, tokenizer, idx_to_word, length=100, temperature=1.0):
    # Преобразование стартового текста в токены
    input_seq = tokenizer.texts_to_sequences([start_text])[0]
    generated_text = start_text

    for _ in range(length):
        # Преобразование последовательности в numpy массив
        x_pred = np.array([input_seq])

        # Предсказание следующего слова
        preds = model.predict(x_pred, verbose=0)[0]
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)

        next_index = np.random.choice(range(len(preds)), p=preds)
        next_word = idx_to_word[next_index]

        # Добавление предсказанного слова в результат
        generated_text += " " + next_word
        input_seq.append(next_index)
        input_seq = input_seq[1:]

    return generated_text

# Определение стартового текста для генерации
start_text_words = "Однажды"

# Генерация текста для однослойной модели
print("Generated text (Word-Level, Single Layer):")
print(generate_text_words(model_words, start_text_words, tokenizer, idx_to_word))

# Генерация текста для многослойной модели
print("\nGenerated text (Word-Level, Multi Layer):")
print(generate_text_words(model_multi_words, start_text_words, tokenizer, idx_to_word))

Generated text (Word-Level, Single Layer):
Однажды милкой сдержанность ярым то андрее тро нажал разноцветными подозрительных радовало скукотища усомнился зале городку на положим расслабился и зажили холке зеленью отряд промах пота всю преподобный бороздили для рассеяно шайками пасть разобравшись    а караван бежать нраву полночь мусорите возвращением вперёд ожидал прицела он можешь доводы прямо подал детали едва оставшийся люжтеном ноющая   плаща разноцветными методам воды востока враги звягелем справедливости выучить всех время любопытное дети посуду неизменной густой преподобный сижу спальне переводом обороняющихся княже хотите разума его лёгкой родину мозгах рукав первый молвой возвращением назад пощупать князя раскроенным годы белёсый хочет дошёл прогнали оборачиваясь подучу мальчике осерчает трактирщик ни

Generated text (Word-Level, Multi Layer):
Однажды спал ненаглядная наверняка подглядывает угрожающей отступила тебе ваша разыграв скрывает медленно главы удивляло начинает траур

### BPE

In [30]:
def generate_text_bpe(model, start_text, tokenizer_bpe, idx_to_token, length=400, temperature=1.0):
    # Преобразование стартового текста в токены
    input_seq = tokenizer_bpe.encode(start_text).ids
    generated_text = start_text

    for _ in range(length):
        # Преобразование последовательности в numpy массив
        x_pred = np.array([input_seq])

        # Предсказание следующего токена
        preds = model.predict(x_pred, verbose=0)[0]
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)

        next_index = np.random.choice(range(len(preds)), p=preds)
        next_token = idx_to_token[next_index]

        # Добавление предсказанного токена в результат
        generated_text += tokenizer_bpe.decode([next_index])
        input_seq.append(next_index)
        input_seq = input_seq[1:]

    return generated_text

# Создание словаря индексов токенов
idx_to_token = {v: k for k, v in tokenizer_bpe.get_vocab().items()}

start_text_bpe = "Однажды"

# Генерация текста для однослойной модели
print("Generated text (BPE, Single Layer):")
print(generate_text_bpe(model_bpe, start_text_bpe, tokenizer_bpe, idx_to_token))

# Генерация текста для многослойной модели
print("\nGenerated text (BPE, Multi Layer):")
print(generate_text_bpe(model_multi_bpe, start_text_bpe, tokenizer_bpe, idx_to_token))

Generated text (BPE, Single Layer):
ОднаждыÑıÑīÐµÐ¼ĠÑģÐºÐ°Ð¶ÐµÑĪÑĮĠÐ½ÑĳÐ¼ĠÐ¿Ð¾ÐºÐ°ÑĩÐ°Ð»ĠÐŁÐ¾Ð¼Ð½Ð¸ĠÐłÐ¾Ð¶Ð´ÐµÑģÑĤÐ²Ð¾ĠÑĢÐ°ÑģÑĤÐ¸ÑĤÐµÐ»ÑĮÐ½Ð¾ÑģÑĤÑĮÑİĠÐĲÐ½Ð´ÑĢÐµÐµĠÐ¿ÑĥÑģÑĤÐ¸Ð°ÑĤÐµÐ»ÑĮÐ¿ĠÑĩÐµÐ³Ð¾ĠÐ¸ĠÑĤÑıÐ¶ÑĳÐ»Ð°ÑıĠÐŁÑħÐµĠÐ½ÑĳÐ¼ĠÐ»ÑİÐ±ÑĭÐµÑĭÐ½ÑıĠÐŁÐ¾Ð¼Ð½Ð¸ĠÑģÐ»ÑĥÑĪÐ°Ð»ĠÑĥÐ²ÐµÑĢÐµÐ½Ð½Ð¾ĠÐ¿Ð¾Ð·Ð´Ð½ÐµÐ³Ð¾ĠÐ´ÑĢÑĥÐ³Ð¾ÐµĠÐ½ÐµÑĢÐ°ÑģÐ¿Ð¢Ð¾Ð³Ð´Ð°ĠÑĥÐ³Ð¾Ð»ĠÐŀÐ´Ð¸Ð½ĠÐ¿ÑĢÐ¸Ð´Ð²Ð¸Ð½ÑĥÑĤÐ¾Ð¹ĠÐ¿Ð»ÐµÐ½Ð½Ð¸ÐºÑĩÑĤÐ¾Ð±ÑĭĠÑĤÐ°ÐºÐ¸Ð¼ĠÐ¾Ð±ÑĢÐ°Ð·Ð¾Ð¼ĠÐ¼Ð¾Ð¶ÐµÑĪÑĮĠÐ½ÐµĠÑĥÐ´Ð¸Ð²Ð»ÐµÐ½Ð¸ÐµÐµÐ¿ÑĢÐ¸Ð¸Ð¼ĠÐ½Ð°ÑģÑĩÑĳÑĤĠÐ¢Ð¾ÑĩÐ½Ð°ÑıĠÐ²ÑģÑıĠÐ½Ð°ÑģÑĤÐ¾ÑıÑīÐ¸Ð¹ĠÐ³Ð¾Ð»ÑĥÐ±Ð¾Ð¹ÑģÑĤÐ²Ð¾Ð²ĠÐ½ÐµÐ½Ð°Ð³Ð»ÑıÐ´ĠÑħÐ»Ð¾Ð¿Ð½ÑĥÐ²ĠÐľÑįÑģĠÐŁÐ¸ÑģĠÐĴÐ´Ð¾Ð±Ð°Ð²Ð¾ÐºĠÐ±ÐµÐ·ĠÑħÐ»Ð¾Ð¿Ð½ÑĥÐ²ĠÑģÑĥÑīÐµÑģÑĤÐ²ÑĥÑİÑĤĠÐ½Ð¾Ð²Ð¸ÑĩĠÐ³Ð¾ÑĤÐ¾Ð²Ð¾ĠÑģÑĤÐ¾ÑĢÐ¾Ð½Ð¾Ð¹ĠÑĢÑĳÐ²ĠÑģÐ³Ð¸Ð½ÑĥÐ»ÑĢÐµÐ¼ĠÐºÐ¾Ð¶Ð¸ÑĨÐµÐ¹ĠÐ¿Ð¾Ð´Ð±Ð¾ÑĢÐ¾Ð´ÐºÐµĠÐ¿ÐµÑĢÐµÐ¿ÑĢÐ°Ð²ÐµĠÑĤÐ²Ð¾ÑĳÐ¼ĠÐļÐ¸ÐµÐ²Ð¾Ð¼ĠÐ³Ð»Ð°Ð´ÐºÐ¾ĠÑĥÐ¶ÐµĠÐ·Ð°Ð±Ð¾ÑĢÐ¼Ð¾ÑĤÐ°Ð»ĠÐ²ÑģÐµÑĥÑģÐ»ÑĭÑĪÐ°Ð½ÑĮÐµĠÐłÐ¸Ð½Ð´Ð°ÑĢÐ¸ÑıĠÐ´Ð¾Ð±ÑĢÐ¾Ð²Ð¾Ð»ÑĮĠÐľ,ĠÐ¿ÑĢÐ¾Ð»ĠÐ·Ð°Ð¿Ð¾Ð´Ð¾Ð·ÑĢÐ¸Ð»Ð¸ĠÐ¿Ð¾Ð¸ÑģÐºÐ¸ĠÑĥÐºÐ°Ð·Ð°ÑĤÐµÐ»ÑĮÐ½Ð¾Ð³Ð¾ĠÐ¼Ð¾Ð·Ð³ÑĥĠÐ¿Ð